In [2]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema import BaseOutputParser

from dotenv import load_dotenv
import os

load_dotenv()  # take environment variables from .env.
api_key = os.getenv("OPENAI_API_KEY")

if api_key is None:
    raise ValueError("OPENAI_API_KEY가 .env 파일에 설정되지 않았습니다.")


chat = ChatOpenAI(
    openai_api_key=api_key,
    temperature=0.1
    )
chat = ChatOpenAI()

class CommaOutputParser(BaseOutputParser):
    def parse(self, text):
        items = text.strip().split(",")
        return list(map(str.strip, items))

template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a list generating machine. Everything you are asked will be answered with a comma separated list of max {max_items} in lowercase.Do NOT reply with anything else.",
        ),
        ("human", "{question}"),
    ]
)

chain = template | chat | CommaOutputParser()

chain.invoke({"max_items": 5, "question": "What are the pokemons?"})

['pikachu', 'bulbasaur', 'charmander', 'squirtle', 'jigglypuff']

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from dotenv import load_dotenv
import os

load_dotenv()  # take environment variables from .env.
api_key = os.getenv("OPENAI_API_KEY")

if api_key is None:
    raise ValueError("OPENAI_API_KEY가 .env 파일에 설정되지 않았습니다.")

chat = ChatOpenAI(
    openai_api_key=api_key,
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

chef_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "",
        ),
        ("human", "I want to cook {cuisine} food."),
    ]
)

chef_chain = chef_prompt | chat
veg_chef_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a vegetarian chef specialized on making traditional recipies vegetarian. You find alternative ingredients and explain their preparation. You don't radically modify the recipe. If there is no alternative for a food just say you don't know how to replace it.",
        ),
        ("human", "{recipe}"),
    ]
)


veg_chain = veg_chef_prompt | chat


final_chain = {"recipe": chef_chain} | veg_chain

final_chain.invoke({"cuisine": "indian"})

That's great! Indian cuisine is known for its rich flavors and variety of dishes. Here are a few popular Indian recipes you can try:

1. Butter Chicken: A creamy and flavorful chicken dish cooked in a rich tomato and butter sauce.

2. Chana Masala: A spicy and tangy chickpea curry made with a blend of aromatic spices.

3. Palak Paneer: A creamy spinach curry with cubes of paneer (Indian cottage cheese) cooked in a flavorful gravy.

4. Biryani: A fragrant and flavorful rice dish cooked with meat or vegetables and a blend of spices.

5. Aloo Gobi: A simple and delicious dish made with potatoes and cauliflower cooked with spices.

6. Dal Tadka: A comforting lentil curry made with yellow lentils tempered with spices.

These are just a few examples of the many delicious Indian dishes you can try cooking at home. Enjoy your culinary adventure!For the Butter Chicken recipe, you can replace the chicken with paneer or tofu to make it vegetarian. Paneer is a common substitute in Indian cuisine a

AIMessage(content='For the Butter Chicken recipe, you can replace the chicken with paneer or tofu to make it vegetarian. Paneer is a common substitute in Indian cuisine and can be found in most grocery stores. To prepare paneer for this dish, you can cut it into cubes and lightly fry or grill them before adding them to the tomato and butter sauce.\n\nIn the Chana Masala recipe, you can simply omit the chicken and increase the amount of chickpeas to make it a hearty vegetarian dish. You can also add some diced potatoes or carrots for extra texture and flavor.\n\nFor Palak Paneer, you can use the same paneer substitute mentioned earlier. To prepare the paneer for this dish, you can lightly fry or grill the paneer cubes before adding them to the creamy spinach gravy.\n\nIn Biryani, you can replace the meat with vegetables like potatoes, carrots, peas, and cauliflower to make a flavorful vegetarian version. You can also add some paneer cubes for a protein boost.\n\nAloo Gobi is already a v